# Scraping many pages + Using Selenium

## The pages we'll be looking at

If I wanted to read specific information about a specfic mine, it takes a few steps. **Do these steps with your browser before you try any programming.**

1. Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm)
2. Scroll down to **Mine Identification Number (ID) Search**
3. Type in a mine ID number, such as `3503598`, click **Search**
4. I'm on a page! It lists the MINE NAME and MINE OWNER.

After searching for and finding a mine, I can use this page to **find reports about this mine**. Some of the reports are on accidents, violations, inspections, health samples and more. To get those reports:

1. Search for a mine (if you haven't already)
2. Scroll down and change **Beginning Date** to `1/1/1995` (violation reports begin in 1995, accidents begin in 1983)
3. Select the report type of `Violations`
4. Click **Get Report**
5. I'm on a page! It lists ALL OF THE MINE'S VIOLATIONS.

By changing the report type you're searching for you can find all sorts of different data.

# Researching mine information

## Preparation 

### When you search for information on a specific mine, what URL should Selenium visit first?

- *TIP: the answer is NOT `https://arlweb.msha.gov/drs/ASP/BasicMineInfonew.asp`*

In [202]:
# https://arlweb.msha.gov/drs/drshome.htm

### How can you identify the text field we're going to type the Mine ID into?

Selenium can find elements by:

- name
- Class
- ID
- CSS selector (**ASK ME WHAT THIS IS** if you don't know)
- XPath (**ASK ME WHAT THIS IS** because you definitely don't know)
- Link text
- Partial link text

So in other words, what's unique about this element?

In [203]:
# from selenium import webdriver
# driver = webdriver.Chrome()
# driver.get("https://arlweb.msha.gov/drs/drshome.htm")

### How can you identify the search button we're going to click, or the form we're going to submit?

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

In [204]:
# search_input = driver.find_elements_by_xpath('//*[@id="inputdrs"]')[1]
# search_input.send_keys('3503598')

In [205]:
# search_box = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
# search_box.click()

### Use Selenium to search using the mine ID `3901432`. Get me the operator's name by scraping.

- *TIP: You can find elements/text using Selenium, or use BeautifulSoup with `doc = BeautifulSoup(driver.page_source)`*

In [206]:
from selenium import webdriver
driver = webdriver.Chrome()
driver.get("https://arlweb.msha.gov/drs/drshome.htm")

In [207]:
search_input = driver.find_elements_by_xpath('//*[@id="inputdrs"]')[1]
search_input.send_keys('3901432')

In [208]:
search_box = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
search_box.click()

In [209]:
opr = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[1]/tbody/tr[3]/td[2]/font/b')
print(opr.text)
driver.close()

Krueger Brothers Gravel & Dirt


# Using .apply to find data about SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

In [210]:
import pandas as pd

### Open up `mines-subset.csv` in a text editor, then look at your dataframe. Is something different about them?

In [211]:
mines_sub_df = pd.read_csv('mines-subset.csv', dtype = {'id': 'str'}) # read as string
mines_sub_df.head()

,id
0,4104757
1,0801306
2,3609931


In [212]:
!cat mines-subset.csv

id
4104757
0801306
3609931

### Scrape the operator's name for each of those mines and print it

- *TIP: use .apply and a function*
- *TIP: If you need help with .apply, look at the "Using apply in pandas" notebook *

In [213]:
def scrape_name(row):
    driver.get("https://arlweb.msha.gov/drs/drshome.htm")
    search_input = driver.find_elements_by_xpath('//*[@id="inputdrs"]')[1]
    search_input.send_keys(row['id'])
    search_box = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
    search_box.click()
    opr = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[1]/tbody/tr[3]/td[2]/font/b')
    print(opr.text)
    print("Mine ID:", row['id'])
    print("------")
    
    
driver = webdriver.Chrome()
mines_sub_df.apply(scrape_name, axis=1) # axis=1, do it for every row
driver.close()

Dirt Works
Mine ID: 4104757
------
Holley Dirt Company, Inc
Mine ID: 0801306
------
M.R. Dirt Inc.
Mine ID: 3609931
------


### Scrape the operator's name and save it into a new column

- *TIP: Use .apply and a function*
- *TIP: Remember to use `return`*

In [214]:
def scrape_name(row):
    driver.get("https://arlweb.msha.gov/drs/drshome.htm")
    search_input = driver.find_elements_by_xpath('//*[@id="inputdrs"]')[1]
    search_input.send_keys(row['id'])
    search_box = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
    search_box.click()
    opr = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[1]/tbody/tr[3]/td[2]/font/b')
    return opr.text

In [215]:
driver = webdriver.Chrome()
mines_sub_df['operator name'] = mines_sub_df.apply(scrape_name, axis=1) # axis=1, do it for every row
driver.close()

In [216]:
mines_sub_df.head()

,id,operator name
0,4104757,Dirt Works
1,0801306,"Holley Dirt Company, Inc"
2,3609931,M.R. Dirt Inc.


# Researching mine violations

Read the very top again to remember how to find mine violations

### When you search for a mine's violations, what URL is Selenium going to start on?

- *TIP: `requests` can send form data to load in the middle of a bunch of steps, but Selenium has to start at the beginning

In [217]:
# https://arlweb.msha.gov/drs/drshome.htm

### When you're searching for violations from the Mine Information page, how are you going to identify the "Beginning Date" field?

In [218]:
# //*[@id="content"]/form[1]/table[2]/tbody/tr[2]/td/font/input[1]

### When you're searching for violations from the Mine Information page, how are you going to identify the "Violations" button?

In [219]:
# //*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input

### When you're searching for violations from the Mine Information page, how are you going to identify the form or the button to click to get a list of the violations?

In [220]:
# //*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input

### Using the mine ID `3901432`, scrape all of their violations since 1/1/1995

**Save this into a CSV called `3901432-violations.csv`.** This CSV must include the following fields:

- Citation number
- Case number
- Standard violated
- Link to standard
- Proposed penalty
- Amount paid to date

**Tips:**

- *TIP: It's probably worth it to print them all first, then save them to a CSV once you know it's all working.*
- *TIP: You'll use the parent pattern - get the ROWS first (tr), then loop through and get the TABLE CELLS (td)*

In [221]:
# get into mine infomation page
driver = webdriver.Chrome()
driver.get("https://arlweb.msha.gov/drs/drshome.htm")
search_input = driver.find_elements_by_xpath('//*[@id="inputdrs"]')[1]
search_input.send_keys("3901432")
search_box = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
search_box.click()

In [222]:
date_input = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[2]/tbody/tr[2]/td/font/input[1]')
date_input.send_keys("1/1/1995")
violation_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input')
violation_button.click()

In [223]:
get_report = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input')
get_report.click()

In [224]:
# Citation number
# Case number
# Standard violated
# Link to standard
# Proposed penalty
# Amount paid to date

In [225]:
from bs4 import BeautifulSoup

In [226]:
doc = BeautifulSoup(driver.page_source, 'html.parser')

In [227]:
violation_rows = doc.find_all('tr', class_= 'drsviols')
len(violation_rows)

18

In [228]:
for row in violation_rows:
    print("i have a violation")
    cells = row.find_all('td')
    print("citation no", cells[2].text)
    print("case no", cells[3].text)
    print("Standard violated", cells[10].a.text)
    print('Link to standard', cells[10].a['href'])
    print("Proposed penalty", cells[11].text)
    print("Amount paid to date", cells[14].text)

i have a violation
citation no 8750964                        
case no 000361866           
Standard violated 
56.18010            
Link to standard http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-vol1/pdf/CFR-2014-title30-vol1-sec56-18010.pdf
Proposed penalty 100.00
Amount paid to date 100.00 
i have a violation
citation no 6426438                        
case no 000260865           
Standard violated 
56.4101             
Link to standard http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-vol1/pdf/CFR-2011-title30-vol1-sec56-4101.pdf
Proposed penalty 100.00
Amount paid to date 100.00 
i have a violation
citation no 6426439                        
case no 000260865           
Standard violated 
56.4201(a)(2)       
Link to standard http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-vol1/pdf/CFR-2011-title30-vol1-sec56-4201.pdf
Proposed penalty 100.00
Amount paid to date 100.00 
i have a violation
citation no 6588189                        
case no 000260865           
Standard violated 
56.14200 

In [229]:
# save to csv: the list of dictionary
row_list = []

for row in violation_rows:
    row_dict = {}
    cells = row.find_all('td')
    row_dict["citation no"] = cells[2].text.strip()
    row_dict["case no"] = cells[3].text.strip()
    row_dict["standard violated"] = cells[10].a.text.strip()
    row_dict['link to standard'] = cells[10].a['href']
    row_dict["proposed penalty"] = cells[11].text.strip()
    row_dict["amount paid to date"] = cells[14].text.strip()
    
    row_list.append(row_dict)

In [230]:
import pandas as pd
df = pd.DataFrame(row_list)
df.head()

,amount paid to date,case no,citation no,link to standard,proposed penalty,standard violated
0,100.00,000361866,8750964,http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,100.00,56.18010
1,100.00,000260865,6426438,http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-...,100.00,56.4101
2,100.00,000260865,6426439,http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-...,100.00,56.4201(a)(2)
3,100.00,000260865,6588189,http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-...,100.00,56.14200
4,100.00,000238554,6588210,http://www.gpo.gov/fdsys/pkg/CFR-2010-title30-...,100.00,50.30(a)


In [231]:
df.to_csv('3901432-violations.csv', index=False)

# Using .apply to save mine data for SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

In [232]:
df = pd.read_csv('mines-subset.csv', dtype= {'id': 'str'})
df.head()

,id
0,4104757
1,0801306
2,3609931


### Scrape the violations for each mine

**Save each mine's violations into separate CSV files.** Each CSV file must include the following fields:

- Citation number
- Case number
- Standard violated
- Link to standard
- Proposed penalty
- Amount paid to date

Make sure you are saving them into **separate files.** It might be nice to name them after the mine id.

- *TIP: Use .apply for this*
- *TIP: Print out the ID before you start scraping. That way you can take that ID and search manually to see if there is anything weird about the results.*
- *TIP: If you need help with .apply, look at the "Using apply in pandas" notebook 
- *TIP: It's probably worth it to print the fields first, then save them to a CSV once you know it's all working.*

In [233]:
import time

In [234]:
def scrape_violation(row):
    print(row['id'])
    
    driver.get("https://arlweb.msha.gov/drs/drshome.htm")
    search_input = driver.find_elements_by_xpath('//*[@id="inputdrs"]')[1]
    search_input.send_keys(row['id'])
    search_box = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
    search_box.click()
    date_input = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[2]/tbody/tr[2]/td/font/input[1]')
    date_input.send_keys("1/1/1995")
    violation_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input')
    violation_button.click()
    get_report = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input')
    get_report.click()
    
    doc = BeautifulSoup(driver.page_source, 'html.parser')
    violation_rows = doc.find_all('tr', class_= 'drsviols')

    row_list = []

    for row in violation_rows:
        row_dict = {}
        cells = row.find_all('td')
        row_dict["citation"] = cells[2].text.strip()
        row_dict["case no"] = cells[3].text.strip()

        a_tag = cells[10].find('a')

        if a_tag:
            row_dict["standard"] = a_tag.text.strip()
            row_dict['link'] = a_tag['href']

        if len(cells) > 14:
            row_dict["penalty"] = cells[11].text.strip()
            row_dict["paid"] = cells[14].text.strip()

        row_list.append(row_dict)

    vio_df = pd.DataFrame(row_list)
    vio_df.to_csv(row['id'] + "-violations.csv", index=False)
    
    print("------------------------")

df.apply(scrape_violation, axis = 1)

4104757
4104757


KeyError: ('id', 'occurred at index 0')

In [ ]:
driver.close()